In [50]:
import cv2
import apriltag
import numpy as np
import yaml
fs = open(("config.yaml"),encoding="UTF-8")
datas = yaml.load(fs,Loader=yaml.FullLoader)

{'TagFamily': 'tag16h5', 'ImagePath': 'tag_image/', 'ImageCount': 30, 'GenTags': [0, 1, 2], 'GenPath': 'gen_tags/', 0: {'File': 'tag16h5_0.png', 'Size': 100, 'Canvas': [1000, 1000], 'Extand_Dot': [[400, 0, 50], [0, 400, 50], [-400, 0, 50], [0, -400, 50]], 'Pixpermm': 10}, 1: {'File': 'tag16h5_1.png', 'Size': 100, 'Canvas': [600, 1000], 'Extand_Dot': [[200, 0, 20], [0, 400, 40], [-200, 0, 20], [0, -400, 40]], 'Pixpermm': 5}, 2: {'File': 'tag16h5_2.png', 'Size': 100, 'Canvas': [600, 600], 'Extand_Dot': [[200, 200, 20], [-200, -200, 20], [-200, 200, 20], [200, -200, 20]], 'Pixpermm': 2}}


In [46]:
for i in range(30):
    img_path = "tag_image/tag16h5_" + str(i) + ".png"
    RGB_IMG = cv2.imread(img_path)
    crop = RGB_IMG[153:663, 153:663]
    resize = cv2.resize(crop, (1000, 1000), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite("new_tag/tag16h5_"+ str(i) +".png", resize)



In [51]:
for i in datas["GenTags"]:
    tag = cv2.imread(datas["ImagePath"] + datas[i]["File"])
    resize = cv2.resize(tag, (datas[i]["Size"] * datas[i]["Pixpermm"] ,datas[i]["Size"] * datas[i]["Pixpermm"]), interpolation=cv2.INTER_NEAREST)
    canvas = np.zeros((datas[i]["Canvas"][0] * datas[i]["Pixpermm"], datas[i]["Canvas"][1] * datas[i]["Pixpermm"], 3),dtype=np.uint8)
    canvas = ~canvas
    start_x = int(datas[i]["Canvas"][0] * datas[i]["Pixpermm"]/2 - datas[i]["Size"] * datas[i]["Pixpermm"]/2)
    end_x = int(datas[i]["Canvas"][0] * datas[i]["Pixpermm"]/2 + datas[i]["Size"] * datas[i]["Pixpermm"]/2)
    start_y = int(datas[i]["Canvas"][1] * datas[i]["Pixpermm"]/2 - datas[i]["Size"] * datas[i]["Pixpermm"]/2)
    end_y = int(datas[i]["Canvas"][1] * datas[i]["Pixpermm"]/2 + datas[i]["Size"] * datas[i]["Pixpermm"]/2)
    canvas[start_x:end_x, start_y:end_y] = resize
    for dot in datas[i]["Extand_Dot"]:
        center_x = int((datas[i]["Canvas"][0]/2 + dot[0]) * datas[i]["Pixpermm"])
        center_y = int((datas[i]["Canvas"][1]/2 + dot[1]) * datas[i]["Pixpermm"])
        radius = dot[2] * datas[i]["Pixpermm"]
        cv2.circle(canvas, (center_y, center_x), radius, (0, 0, 0), -1)
    cv2.imwrite(datas["GenPath"] + datas[i]["File"], canvas)
    
    

In [68]:
img_path = r'/Users/yuxuan/Documents/test/1.png'
RGB_IMG = cv2.imread(img_path)
Fliped = np.zeros((480, 640, 3),dtype=np.uint8)
cv2.flip(RGB_IMG, 0, Fliped)
GaryImg = cv2.cvtColor(Fliped, cv2.COLOR_BGR2GRAY)
# cv2.imshow('test',GaryImg)
# cv2.waitKey(5000)

In [69]:
BlurGary = cv2.GaussianBlur(GaryImg, (7, 7), 1.2)
cv2.imwrite("/Users/yuxuan/Documents/test/GaussianBlur_2.png", BlurGary)
cv2.imshow('test',BlurGary)
cv2.waitKey(1000)

options = apriltag.DetectorOptions(families="tag16h5",nthreads=1)
detector = apriltag.Detector(options)
results = detector.detect(BlurGary)

In [4]:
results

[Detection(tag_family=b'tag16h5', tag_id=0, hamming=0, goodness=0.0, decision_margin=85.83958435058594, homography=array([[-3.51779560e-03, -4.34790321e-01, -1.73754450e+01],
        [ 7.80618568e-01,  9.00301379e-02, -1.35020163e+01],
        [ 3.05824414e-04,  7.60322828e-04, -5.06531687e-02]]), center=array([343.02779897, 266.55817679]), corners=array([[327.48184204, 277.89743042],
        [331.53875732, 250.67526245],
        [359.24224854, 254.73132324],
        [354.72488403, 282.72869873]]))]

In [65]:
for r in results:
	# extract the bounding box (x, y)-coordinates for the AprilTag
	# and convert each of the (x, y)-coordinate pairs to integers
	(ptA, ptB, ptC, ptD) = r.corners
	ptB = (int(ptB[0]), int(ptB[1]))
	ptC = (int(ptC[0]), int(ptC[1]))
	ptD = (int(ptD[0]), int(ptD[1]))
	ptA = (int(ptA[0]), int(ptA[1]))
	# draw the bounding box of the AprilTag detection
	cv2.line(Fliped, ptA, ptB, (0, 255, 0), 2)
	cv2.line(Fliped, ptB, ptC, (0, 255, 0), 2)
	cv2.line(Fliped, ptC, ptD, (0, 255, 0), 2)
	cv2.line(Fliped, ptD, ptA, (0, 255, 0), 2)
	# draw the center (x, y)-coordinates of the AprilTag
	(cX, cY) = (int(r.center[0]), int(r.center[1]))
	cv2.circle(Fliped, (cX, cY), 5, (0, 0, 255), -1)
	# draw the tag family on the image
	tagFamily = r.tag_family.decode("utf-8")
	# cv2.putText(Fliped, tagFamily, (ptA[0], ptA[1] - 15),
	# 	cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
	print("[INFO] tag family: {}".format(tagFamily))
# show the output image after AprilTag detection
cv2.imwrite("/Users/yuxuan/Documents/test/Tag_1.png", Fliped)
cv2.imshow("Image", Fliped)
cv2.waitKey(0)



[INFO] tag family: tag16h5


115

In [66]:
import math
dis = (results[0].corners[0][0] - results[0].corners[1][0]) ** 2 + (results[0].corners[0][1] - results[0].corners[1][1]) ** 2 +\
    (results[0].corners[2][0] - results[0].corners[1][0]) ** 2 + (results[0].corners[2][1] - results[0].corners[1][1]) ** 2 +\
    (results[0].corners[2][0] - results[0].corners[3][0]) ** 2 + (results[0].corners[2][1] - results[0].corners[3][1]) ** 2 +\
    (results[0].corners[0][0] - results[0].corners[3][0]) ** 2 + (results[0].corners[0][1] - results[0].corners[3][1]) ** 2
side = math.sqrt(dis / 4) 

(ptA, ptB, ptC, ptD) = results[0].corners
center = np.array(results[0].center)

ptA = np.array(ptA)
ptB = np.array(ptB)
ptC = np.array(ptC)
ptD = np.array(ptD)
Side_A = (ptB - ptA + ptC - ptD) / 2
Side_B = (ptB + ptA - ptC - ptD) / 2
circle_1 = center - 8/5 * Side_A
circle_2 = center + 8/5 * Side_A
circle_3 = center - 8/5 * Side_B
circle_4 = center + 8/5 * Side_B

cv2.circle(Fliped, (int(circle_1[0]), int(circle_1[1])), 25, (0, 0, 255), 2)
cv2.circle(Fliped, (int(circle_2[0]), int(circle_2[1])), 25, (0, 0, 255), 2)
cv2.circle(Fliped, (int(circle_3[0]), int(circle_3[1])), 25, (0, 0, 255), 2)
cv2.circle(Fliped, (int(circle_4[0]), int(circle_4[1])), 25, (0, 0, 255), 2)

cv2.imwrite("/Users/yuxuan/Documents/test/Range_1.png", Fliped)
cv2.imshow("Image", Fliped)
cv2.waitKey(0)


115

In [67]:
serach_area = int (side / 2)


thres = 100
for c in [circle_1, circle_2, circle_3, circle_4]:
    point_x =[]
    point_y = []
    for i in range(int(c[0]) - serach_area, int(c[0]) + serach_area + 1):
        for j in range(int(c[1]) - serach_area, int(c[1]) + serach_area + 1):
            if  BlurGary[j][i] < thres:
                point_x.append(i)
                point_y.append(j)
    cv2.circle(Fliped, (int(np.array(point_x).mean()), int(np.array(point_y).mean())), 2, (0, 255, 0), 2)

cv2.imwrite("/Users/yuxuan/Documents/test/Refined_1.png", Fliped)
cv2.imshow("Image", Fliped)
cv2.waitKey(0)


115

In [47]:
BlurGary.shape

(480, 640)